### Introduction to Neuron & Layers in Pytorch

In [ ]:
import torch
import torchinfo

In [ ]:
# Two Most Important Layers in Neural Networks
torch.nn.Linear(in_features = 10, out_features = 4)
torch.nn.Conv2d(in_channels = 10, out_channels = 4 , kernel_size = (3,3) )

Conv2d(10, 4, kernel_size=(3, 3), stride=(1, 1))

In [ ]:
neuron = torch.nn.Linear(in_features = 7, out_features = 1)

In [ ]:
single_neuron = torch.nn.Linear(in_features = 7, out_features = 1)
two_neurons   = torch.nn.Linear(in_features = 7, out_features = 2)

In [ ]:
neuron

Linear(in_features=7, out_features=1, bias=True)

In [ ]:
# incoming_features =
hidden_layer_4neurons_7incoming_connections = torch.nn.Linear(in_features = 7, out_features = 4)
layer = torch.nn.Linear(in_features = 7, out_features = 4)

In [ ]:
neuron.weight , neuron.bias

(Parameter containing:
 tensor([[-0.3680,  0.1754, -0.3521,  0.0451, -0.2955, -0.1238,  0.1055]],
        requires_grad=True),
 Parameter containing:
 tensor([0.3220], requires_grad=True))

In [ ]:
layer.weight

Parameter containing:
tensor([[-0.0210, -0.1476, -0.2578,  0.1311, -0.0995, -0.0612, -0.0658],
        [ 0.3667,  0.2084,  0.0590,  0.2777, -0.1867, -0.1898,  0.1450],
        [-0.2967,  0.2312, -0.1104,  0.3452,  0.2795, -0.0177,  0.1001],
        [-0.1783,  0.1250, -0.0018, -0.0288,  0.3249, -0.1576, -0.1186]],
       requires_grad=True)

#### Class Assignment

1. Build a Neural Network of 3 Layers [4, 5, 2] in Tensorflow Playground & NN SVG
2. Build the same Neural Network in Pytorch Code as well
3. Print weights of 3rd Neuron in 2nd Layer
4. Print weights of 1st Neuron in 3rd Layer

**When in doubt, use two universal functions in python, vars(SOMETHING) and dir(SOMETHING)**

In [ ]:
layer_2 = torch.nn.Linear(in_features = 4, out_features = 5)
layer_3 = torch.nn.Linear(in_features = 5, out_features = 2)

In [ ]:
layer_2.weight[3]

tensor([ 0.2262,  0.2493, -0.4057,  0.2504], grad_fn=<SelectBackward0>)

In [ ]:
vars(layer_2)

{'training': True,
 '_parameters': OrderedDict([('weight', Parameter containing:
               tensor([[-0.2025, -0.4718, -0.2250,  0.2422],
                       [ 0.2023, -0.1723,  0.4463, -0.3200],
                       [ 0.3479,  0.0548,  0.4937,  0.3608],
                       [ 0.2262,  0.2493, -0.4057,  0.2504],
                       [ 0.4524, -0.2140,  0.2721,  0.2073]], requires_grad=True)),
              ('bias',
               Parameter containing:
               tensor([ 0.0582,  0.3151, -0.1962, -0.1422, -0.4621], requires_grad=True))]),
 '_buffers': OrderedDict(),
 '_non_persistent_buffers_set': set(),
 '_backward_pre_hooks': OrderedDict(),
 '_backward_hooks': OrderedDict(),
 '_is_full_backward_hook': None,
 '_forward_hooks': OrderedDict(),
 '_forward_hooks_with_kwargs': OrderedDict(),
 '_forward_hooks_always_called': OrderedDict(),
 '_forward_pre_hooks': OrderedDict(),
 '_forward_pre_hooks_with_kwargs': OrderedDict(),
 '_state_dict_hooks': OrderedDict(),
 '_state_di

In [ ]:
import datasets     as huggingface_datasets

dataset            = huggingface_datasets.load_dataset("mnist",) # streaming = True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/60000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [ ]:
dataset['train'][0]

{'image': <PIL.PngImagePlugin.PngImageFile image mode=L size=28x28>,
 'label': 5}

In [ ]:
dataset['train'][10]['image']

### Downloading the Dataset

In [ ]:
!pip install datasets torchinfo wandb torchmetrics datasets[vision]

In [ ]:
# ! KEY CODE
import datasets
import datasets     as huggingface_datasets
import torch, torchvision, torchinfo,
import torch.nn as nn
import torchmetrics
import wandb
from tqdm import tqdm
from torch.nn import ReLU as Threshold_Filter

def GET_DATASETS():
    # Download
    dataset            = huggingface_datasets.load_dataset("mnist", ) # streaming = True)
    training_dataset   = dataset['train']
    validation_dataset = dataset['test']

    # Transform
    transformations_list = torchvision.transforms.Compose([
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Resize(7,7),
    ])

    def transform_datasets(examples):
        examples["converted_tensors"] = []

        for image in examples['image']:
            transformed_image = transformations_list(image)
            examples['converted_tensors'].append(transformed_image)

        return examples

    training_dataset       = training_dataset   .map(transform_datasets  , batched= True)
    validation_dataset     = validation_dataset .map(transform_datasets  , batched= True)

    # Convert
    new_training_dataset   = training_dataset   .with_format("torch", columns=['label', 'converted_tensors'], dtype = torch.float32)
    new_validation_dataset = validation_dataset .with_format("torch", columns=['label', 'converted_tensors'], dtype = torch.float32)

    return new_training_dataset, new_validation_dataset

training_dataset, validation_dataset = GET_DATASETS()

BATCH_SIZE    = 4
TOTAL_BATCHES = len(training_dataset) / BATCH_SIZE

training_dataloader   = torch.utils.data.DataLoader( dataset= training_dataset   , batch_size= BATCH_SIZE, shuffle= True )
validation_dataloader = torch.utils.data.DataLoader( dataset= validation_dataset , batch_size= BATCH_SIZE, shuffle= True )

### Writing Model

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# [ Input Layer = 28*28*1 , Hidden Layer 1 = 20, Hidden Layer 2 = 15, Hidden Layer 3 = 10 ]
randomly_initialized_model = torch.nn.Sequential(
    nn.Flatten(start_dim=1),

    nn.Linear(in_features = 28*28*1  , out_features = 20   ), Threshold_Filter(),                 # LAYER 1: 1st Hidden Layer
    nn.Linear(in_features = 20       , out_features = 15   ), Threshold_Filter(),                 # LAYER 2: 2nd Hidden Layer

    nn.Linear(in_features = 15       , out_features = 10   ),                            # OUTPUT LAYER
)

randomly_initialized_model_accelerated = randomly_initialized_model.to(device)

model = randomly_initialized_model_accelerated


torchinfo.summary(model, input_size=(1,28,28), verbose = 2);

Layer (type:depth-idx)                   Output Shape              Param #
Sequential                               [1, 10]                   --
├─Flatten: 1-1                           [1, 784]                  --
├─Linear: 1-2                            [1, 20]                   15,700
│    └─weight                                                      ├─15,680
│    └─bias                                                        └─20
├─ReLU: 1-3                              [1, 20]                   --
├─Linear: 1-4                            [1, 15]                   315
│    └─weight                                                      ├─300
│    └─bias                                                        └─15
├─ReLU: 1-5                              [1, 15]                   --
├─Linear: 1-6                            [1, 10]                   160
│    └─weight                                                      ├─150
│    └─bias                                                    

### Model Architecture Experimentation

Half an Hour Lab
### Question 1: Which Config is better? And Why?**[40, 30, 20 , 10, 10] vs [10, 20, 30 , 40, 10]** for MNIST Data
- In Tensorflow Playground, configuration is [8, 4, 2] vs [2 , 4 , 8 ]

In [ ]:
Neural_Network_config = [256, 256, 512, 512]

layers = []
layers.append( nn.Flatten(start_dim=1) )
prev   = 1*28*28
for current_neurons in Neural_Network_config:
    layers.append(    nn.Linear(prev , out_features = current_neurons)    )
    layers.append(    Threshold_Filter()   )
    prev = current_neurons

layers.pop(len(layers) - 1 ) # Final Layer doesn't have activation function. It is a value we need called LOGITS

model = nn.Sequential ( * layers )
model

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=10, bias=True)
  (2): ReLU()
  (3): Linear(in_features=10, out_features=20, bias=True)
  (4): ReLU()
  (5): Linear(in_features=20, out_features=30, bias=True)
  (6): ReLU()
  (7): Linear(in_features=30, out_features=40, bias=True)
  (8): ReLU()
  (9): Linear(in_features=40, out_features=10, bias=True)
)

In [ ]:
torchinfo.summary(model, input_size = (1,28,28))

Layer (type:depth-idx)                   Output Shape              Param #
Sequential                               [1, 10]                   --
├─Flatten: 1-1                           [1, 784]                  --
├─Linear: 1-2                            [1, 40]                   31,400
├─ReLU: 1-3                              [1, 40]                   --
├─Linear: 1-4                            [1, 30]                   1,230
├─ReLU: 1-5                              [1, 30]                   --
├─Linear: 1-6                            [1, 20]                   620
├─ReLU: 1-7                              [1, 20]                   --
├─Linear: 1-8                            [1, 10]                   210
├─ReLU: 1-9                              [1, 10]                   --
├─Linear: 1-10                           [1, 10]                   110
Total params: 33,570
Trainable params: 33,570
Non-trainable params: 0
Total mult-adds (M): 0.03
Input size (MB): 0.00
Forward/backward pass size 

In [ ]:
model[3].weight[15 - 1]

tensor([ 0.1152, -0.1091,  0.1802, -0.1192,  0.1874, -0.0440, -0.0903,  0.1524,
        -0.0151, -0.0959, -0.1603,  0.1062,  0.1851, -0.0507, -0.0870,  0.1629,
        -0.1124, -0.0665, -0.1739, -0.1646, -0.1797,  0.0321, -0.0490,  0.0900,
         0.0693,  0.0327,  0.0551, -0.1290], grad_fn=<SelectBackward0>)

In [ ]:
layer_number_natural = 2
model[layer_number_natural - 1].weight, model[layer_number_natural - 1].bias

(Parameter containing:
 tensor([[ 0.0223,  0.0204, -0.0140,  ..., -0.0018,  0.0098, -0.0277],
         [-0.0040, -0.0214, -0.0120,  ...,  0.0338,  0.0187,  0.0305],
         [-0.0340,  0.0305, -0.0132,  ..., -0.0161,  0.0110, -0.0259],
         ...,
         [ 0.0063, -0.0147, -0.0170,  ...,  0.0010,  0.0146,  0.0237],
         [ 0.0128,  0.0352, -0.0254,  ...,  0.0028, -0.0040,  0.0219],
         [ 0.0222, -0.0199,  0.0038,  ..., -0.0315, -0.0353,  0.0333]],
        requires_grad=True),
 Parameter containing:
 tensor([ 0.0052, -0.0348,  0.0227, -0.0244, -0.0072,  0.0142,  0.0191, -0.0064,
         -0.0219,  0.0279,  0.0347,  0.0195, -0.0062, -0.0010, -0.0074,  0.0073,
          0.0158,  0.0017, -0.0080, -0.0356], requires_grad=True))

## Training Model

In [ ]:
def TRAIN_MODEL(model, training_dataloader, validation_dataloader):

    # 1. Model Parameters & Their Relationship with Error
    parameters_to_optimize = list(model.parameters())

    torch.optim      # Optimizer to minimize Loss
    LEARNING_RATE    = 0.001
    OPTIMIZER        = torch.optim.SGD( params= parameters_to_optimize, lr= LEARNING_RATE )   # 2. Registering Model Parameters with Optimizer, as variables to be minimized

    # 4. Calculation of Error of Prediction
    ERROR_FUNC       = nn.functional.cross_entropy

    # Example of Partial Derivatie
    torch.autograd.grad
    # x_actual, y_actual = 0, 0
    # dError_dWeights  = torch.autograd.grad(
                                            # outputs = ERROR_FUNC( model(x_actual), y_actual ),
                                            # inputs = parameters_to_optimize
                                            # )

    # 5. Calculation of relationship of Error & Parameters
    model.train(mode=True)

    wandb.init()
    epochs = 10
    for epoch_no in range(epochs):

        metric = torchmetrics.Accuracy(task="multiclass", num_classes=10)

        for batch_no, batch_dictionary in enumerate(tqdm(training_dataloader)):
            x_actual = batch_dictionary['converted_tensors']
            y_actual = batch_dictionary['label']

            x_actual = x_actual.to(device)
            y_actual = y_actual.to(device)

            y_predicted_LOGITS = model.forward(x_actual)
            loss               = ERROR_FUNC(y_predicted_LOGITS, y_actual.to(torch.int64))
            y_predicted_probs  = nn.functional.softmax(y_predicted_LOGITS, dim= 1)


            OPTIMIZER.zero_grad()
            loss.backward()
            OPTIMIZER.step()
            """
            loss.backward() => dError_dWeights  = torch.autograd.grad( outputs = ERROR_FUNC( model(x_actual), y_actual ), inputs = parameters_to_optimize)
            for (name, weight), gradient in zip(model.named_parameters(), dError_dWeights):
                print(name)
                weight = weight - gradient * LEARNING_RATE

                print("weight & gradient shape = ", weight.shape, gradient.shape)
                print("weight & gradient values= ", weight, gradient)
                print("non zero gradients", torch.count_nonzero(gradient))
            """

            loss_batch      = loss.item()
            accuracy_batch  = metric(y_predicted_LOGITS, y_actual)
            training_accuracy_avg_epoch = metric.compute() # calculates average accuracy across epoch automatically

            metrics_per_batch = {
                "loss": loss_batch,
                # "accuracy_batch": accuracy_batch,
                "batch_no": batch_no,
                "training_accuracy_avg_epoch": training_accuracy_avg_epoch
            }
            wandb.log(metrics_per_batch)

        metric.reset()
        evaluate_model(model, validation_dataloader, epoch_no, training_accuracy_avg_epoch)

    # loss.backward(), computes dloss/dw for every parameter w which has requires_grad=True.
def evaluate_model(model, validation_dataloader, epoch_no, training_accuracy_avg_epoch):
    model.eval()

    metric = torchmetrics.Accuracy(task="multiclass", num_classes=10)

    with torch.no_grad():
        for batch_no, batch_dictionary in enumerate(tqdm(training_dataloader)):
            x_actual = batch_dictionary['converted_tensors']
            y_actual = batch_dictionary['label']

            x_actual = x_actual.to(device)
            y_actual = y_actual.to(device)

            y_predicted_LOGITS = model.forward(x_actual)
            accuracy_batch     = metric(y_predicted_LOGITS, y_actual)

        testing_accuracy_avg = metric.compute()

        metrics_per_epoch = {
            "training_accuracy_avg_epoch": training_accuracy_avg_epoch,
            "testing_accuracy_avg": testing_accuracy_avg,
            "epoch_no": epoch_no
        }
        wandb.log(metrics_per_epoch)



TRAIN_MODEL(model, training_dataloader, validation_dataloader)


batch_no,▁▂▃▄▅▇█▁▂▃▅▆▇█▁▃▄▅▆▇▁▂▃▄▅▇█▁▂▃▅▆▇█▁▃▄▅▆█
epoch_no,▁▃▅▆█
loss,▅▅▅▅▅▅▅▅▅▅▄▅▄▄▄█▄▄▂▁▆▂▂▂▁▁▂▁▁▂▂▁▂▂▁▁▁▁▂▂
testing_accuracy_avg,▁▃▇██
training_accuracy_avg_epoch,▁▁▁▁▁▁▁▂▂▂▂▂▂▂▄▄▅▅▅▆▇▇▇▇▇▇▇█████████████
batch_no,14999
epoch_no,4
loss,0.41915
testing_accuracy_avg,0.90392
training_accuracy_avg_epoch,0.91028


100%|██████████| 15000/15000 [01:13<00:00, 205.33it/s]


In [ ]:
import numpy as np

def save_predictions_csv(y_predicted_probs):
    y_predicted_class = np.argmax(y_predicted_probs, axis=0)
    y_indexes         = np.arange(0, len(y_predicted_class), 1)
    csv_file = open("predictions.csv","w")
    csv_file.write("ImageId,Label")
    np.savetxt(csv_file, np.concatenate((y_indexes, y_predicted_class)), delimiter=",")